Generación Automática de Respuestas a las preguntas que ya tenemos.

In [ ]:
import pandas as pd
from pprint import pprint

Mounted at /content/drive


In [ ]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 4.9

In [ ]:
%pip install llama-index-llms-openai

In [ ]:
import openai
import os

openai.api_key="your-token"

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Response,
)
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd

pd.set_option("display.max_colwidth", 0)

In [ ]:
from llama_index.core.evaluation import EvaluationResult


# define jupyter display function
def display_eval_df(response: Response, eval_result: EvaluationResult) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
            "Reasoning": eval_result.feedback,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    #display(eval_df)
    return eval_df

In [ ]:
reader_or = SimpleDirectoryReader("TodoManual")
documents= reader_or.load_data()
splitter = SentenceSplitter(chunk_size=512)
vector_index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter]
)

ValueError: Directory /content/drive/MyDrive/Colab Notebooks/TFM - RAG/txt_pruebas_llama/TodoManual does not exist.

# GPT 3.5 turbo

In [ ]:
import openai
import pandas as pd
# Configurar la API de OpenAI con tu clave
openai.api_key = 'your-token'

# Configurar el modelo GPT-4 Mini
gpt35_model = "gpt-3.5-turbo"

In [ ]:
gpt35_100= pd.read_csv('traduccion_manual0-100.csv')
gpt35_300= pd.read_csv('traduccion_manual100-300.csv')
gpt35_500= pd.read_csv('traduccion_manual300-500.csv')
gpt35_600= pd.read_csv('traduccion_manual500-600.csv')

In [ ]:
gpt35 = pd.concat([gpt35_100, gpt35_300,gpt35_500,gpt35_600])

In [ ]:
gpt35['reference_contexts'] = gpt35['reference_contexts'].apply(lambda x: x.replace('[','').replace(']','').replace("'",""))

In [ ]:
gpt35 = gpt35[['reference_contexts','traduccion']]

In [ ]:
gpt35.head()

,reference_contexts,traduccion
0,"4.º El calendario debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más.",¿Cuál es la fecha límite para que el calendario sea impreso y esté disponible para la venta al público?
1,"4.º El calendario debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más.",¿Cuánto debería costar cada copia impresa del calendario si contiene dos hojas?
2,"4.º El calendario debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más.",¿Cuál es el precio de un calendario con solo una hoja según la información proporcionada?
3,"En este discurso dedicado al Supremo Consejo y a cuya publicación precedió la vista de los tres Señores Fiscales, se habla de la importancia de las Leyes formularias, que son acusadas generalmente de ser vacías y supersticiosas. Se mencionan los daños que causan las leyes bien dictadas, pero que no tienen fórmulas definidas para los actos que se tratan. También se demuestran los riesgos de contradicción y los innumerables beneficios que se obtendrán aplicando el formulario de los juicios, evitando la prolongación de los juicios y reduciendo el número de abogados y otras personas que trabajan en el Foro.",¿Por qué se menciona la importancia de las Leyes formularias en el discurso dirigido al Supremo Consejo? \n\n¿Por qué se menciona la importancia de las Leyes formularias en el discurso dirigido al Supremo Consejo?
4,"En este discurso dedicado al Supremo Consejo y a cuya publicación precedió la vista de los tres Señores Fiscales, se habla de la importancia de las Leyes formularias, que son acusadas generalmente de ser vacías y supersticiosas. Se mencionan los daños que causan las leyes bien dictadas, pero que no tienen fórmulas definidas para los actos que se tratan. También se demuestran los riesgos de contradicción y los innumerables beneficios que se obtendrán aplicando el formulario de los juicios, evitando la prolongación de los juicios y reduciendo el número de abogados y otras personas que trabajan en el Foro.",¿Cuáles son los daños que se mencionan en el discurso que pueden causar las leyes bien dictadas pero que no tienen fórmulas definidas para los actos que se tratan? \n\n¿Cuáles son los daños que se mencionan en el discurso que pueden causar las leyes bien dictadas pero que no tienen fórmulas definidas para los actos que se tratan?


In [ ]:
gpt35 = gpt35.dropna()

In [ ]:
def gen_respuesta(contexto,pregunta):
  query_engine = vector_index.as_query_engine(response_mode="compact")

  response = query_engine.query(f"Dado este contexto: {contexto} responda a la pregunta {pregunta} en español")

  pregunta = response.response#display_response(response)

  return pregunta

In [ ]:
gpt35 = gpt35.reset_index(drop=True )

In [ ]:
from llama_index.core import VectorStoreIndex
reader = SimpleDirectoryReader("Manual0-100")
documents = reader.load_data()
vector_index = VectorStoreIndex.from_documents(documents)

In [ ]:
gpt35_aux = gpt35.head(1)

In [ ]:
gpt35_aux

,reference_contexts,traduccion,respuestas
0,"4.º El calendario debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más.",¿Cuál es la fecha límite para que el calendario sea impreso y esté disponible para la venta al público?,La fecha límite para que el calendario sea impreso y esté disponible para la venta al público es el 1 de noviembre del presente año.


In [ ]:
gpt35['respuestas'] = gpt35.apply(lambda x: gen_respuesta(x['reference_contexts'],x['traduccion']),axis=1)

In [ ]:
gpt35.to_csv('respuestas_gpt35.csv')

# GPT 4o mini

In [ ]:
gpt4_100= pd.read_csv('questions_manual0-100_gpt4-mini.csv')
gpt4_300= pd.read_csv('questions_manual100-300_gpt4-mini.csv')
gpt4_500= pd.read_csv('questions_manual300-500_gpt4-mini.csv')
gpt4_600= pd.read_csv('questions_manual500-600_gpt4-mini.csv')

In [ ]:
gpt4 = pd.concat([gpt4_100, gpt4_300,gpt4_500,gpt4_600])

In [ ]:
gpt4['reference_contexts'] = gpt4['reference_contexts'].apply(lambda x: x.replace('[','').replace(']','').replace("'",""))

In [ ]:
gpt4 = gpt4[['reference_contexts','query']]

In [ ]:
gpt4 = gpt4.dropna()
gpt4 = gpt4.reset_index(drop=True )

In [ ]:
import openai
import pandas as pd
from llama_index.core import Settings
# Configurar la API de OpenAI con tu clave
openai.api_key = 'your-token'


# Configurar el modelo GPT-4 Mini
gpt4_model = "gpt-4o-mini"
Settings.llm = OpenAI(temperature=0, model=gpt4_model)

In [ ]:
reader_or = SimpleDirectoryReader("Manual0-100")
#/content/drive/MyDrive/MierdasCelina/txt_pruebas_llama/Original0-100")
documents= reader_or.load_data()
splitter = SentenceSplitter(chunk_size=512)
vector_index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter]
)

In [ ]:
def gen_respuesta(contexto,pregunta):
  query_engine = vector_index.as_query_engine(response_mode="compact")

  response = query_engine.query(f"Dado este contexto: {contexto} responda a la pregunta {pregunta} en español")

  pregunta = response.response#display_response(response)

  return pregunta

In [ ]:
gpt4_aux = gpt4.head(100)

In [ ]:
gpt4['respuestas'] = gpt4.apply(lambda x: gen_respuesta(x['reference_contexts'],x['query']),axis=1)

In [ ]:
gpt4.head()

,reference_contexts,query,respuestas
0,"4.º El calendario debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más.",¿Cuál es la fecha límite para la impresión y venta del calendario al público según el texto?,La fecha límite para la impresión y venta del calendario al público es el 1 de noviembre de este año.
1,"4.º El calendario debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más.",¿A qué precio debe venderse cada ejemplar impreso de dos pliegos del calendario?,Cada ejemplar impreso de dos pliegos del calendario debe venderse al precio de un real de vellón.
2,"4.º El calendario debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más.","Según el documento, ¿cuál es el precio establecido para el calendario que está impreso en un solo pliego?",El precio establecido para el calendario que está impreso en un solo pliego es de siete cuartos.
3,"En este discurso dedicado al Supremo Consejo y a cuya publicación precedió la vista de los tres Señores Fiscales, se habla de la importancia de las Leyes formularias, que son acusadas generalmente de ser vacías y supersticiosas. Se mencionan los daños que causan las leyes bien dictadas, pero que no tienen fórmulas definidas para los actos que se tratan. También se demuestran los riesgos de contradicción y los innumerables beneficios que se obtendrán aplicando el formulario de los juicios, evitando la prolongación de los juicios y reduciendo el número de abogados y otras personas que trabajan en el Foro.",¿Cuáles son las principales críticas que se mencionan sobre las Leyes formularias en el discurso dirigido al Supremo Consejo?,"Las principales críticas mencionadas sobre las Leyes formularias en el discurso son que son acusadas de ser vanas y supersticiosas. Además, se señala que incluso las leyes bien dictadas pueden causar daños si no cuentan con fórmulas definidas para los actos que regulan. También se destacan los riesgos de contradicción que presentan estas leyes."
4,"En este discurso dedicado al Supremo Consejo y a cuya publicación precedió la vista de los tres Señores Fiscales, se habla de la importancia de las Leyes formularias, que son acusadas generalmente de ser vacías y supersticiosas. Se mencionan los daños que causan las leyes bien dictadas, pero que no tienen fórmulas definidas para los actos que se tratan. También se demuestran los riesgos de contradicción y los innumerables beneficios que se obtendrán aplicando el formulario de los juicios, evitando la prolongación de los juicios y reduciendo el número de abogados y otras personas que trabajan en el Foro.",¿Qué riesgos se identifican en relación con las leyes bien dictadas que carecen de fórmulas definidas para los actos que se tratan?,Se identifican riesgos de contradicción en relación con las leyes bien dictadas que carecen de fórmulas definidas para los actos que se tratan. Esto implica que la falta de claridad y especificidad en las leyes puede llevar a interpretaciones diferentes y conflictos en su aplicación.


In [ ]:
gpt4.to_csv('respuestas_gpt4.csv')

# Mistral

In [ ]:
%pip install llama-index
%pip install transformers accelerate bitsandbytes
%pip install llama-index-readers-web
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface
%pip install llama-index-program-openai
%pip install llama-index-agent-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.2/180.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing inst

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import openai
import os

openai.api_key="your-token"

print(openai.api_key)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from llama_index.core import SimpleDirectoryReaderç

reader = SimpleDirectoryReader("Manual0-100")
documents = reader.load_data()
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)
from llama_index.core.indices import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)
from llama_index.core.response.notebook_utils import display_response
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
mistral = pd.read_csv('preguntas_mistral_original_manual.csv')

In [ ]:
mistral = mistral.rename(columns={'manual':'frase','Preguntas_manual':'pregunta_generada'})[[ "frase","pregunta_generada"]]
mistral["pregunta_generada"] = mistral["pregunta_generada"].apply(lambda x: str(x).replace('? ¿','? \n ¿') )

In [ ]:
# Dividir la columna en filas manteniendo las otras columnas
corpus_mixtral_expanded = mistral.set_index(['frase'])['pregunta_generada'].str.split('\n').apply(pd.Series).stack().reset_index()

# Renombrar las columnas resultantes
corpus_mixtral_expanded.columns = ['frase','level', 'pregunta_generada']

In [ ]:
corpus_mixtral_expanded["pregunta_generada"] = corpus_mixtral_expanded["pregunta_generada"].apply(lambda x: str(x).replace('1. ','').replace('2. ','').replace('3. ','') )

In [ ]:
corpus_mixtral_expanded = corpus_mixtral_expanded.drop('level',axis=1)

In [ ]:
corpus_mixtral_expanded.head()

,frase,pregunta_generada
0,4.º El calendario debe ser impreso y estar a l...,¿Cuál es la fecha límite para que el Almanak e...
1,4.º El calendario debe ser impreso y estar a l...,¿A qué precio se debe vender cada ejemplar imp...
2,4.º El calendario debe ser impreso y estar a l...,¿Cuál será la consecuencia si el Almanak no es...
3,En este discurso dedicado al Supremo Consejo y...,¿Qué se discute en el discurso dirigido al Sup...
4,En este discurso dedicado al Supremo Consejo y...,¿Cuáles son los daños que se mencionan en el t...


In [ ]:
def generate_answer(context, question):
    prompt = prompt = f"""
      Context information is below.
      ---------------------
      {context}
      ---------------------
      Given the context information and not prior knowledge, answer the query in Spanish.
      Query: {question}
      Answer:
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=500)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
corpus_mixtral_expanded_10 = corpus_mixtral_expanded.head(10)

In [ ]:
corpus_mixtral_expanded_10


,frase,pregunta_generada
0,4.º El calendario debe ser impreso y estar a l...,¿Cuál es la fecha límite para que el Almanak e...
1,4.º El calendario debe ser impreso y estar a l...,¿A qué precio se debe vender cada ejemplar imp...
2,4.º El calendario debe ser impreso y estar a l...,¿Cuál será la consecuencia si el Almanak no es...
3,En este discurso dedicado al Supremo Consejo y...,¿Qué se discute en el discurso dirigido al Sup...
4,En este discurso dedicado al Supremo Consejo y...,¿Cuáles son los daños que se mencionan en el t...
5,En este discurso dedicado al Supremo Consejo y...,¿Qué beneficios se destacan al aplicar el form...
6,"El día 16 del mes pasado, entre las 12:30 y la...",¿Qué raza de perro se perdió el día 16 del mes...
7,"El día 16 del mes pasado, entre las 12:30 y la...",¿De qué color era el cachorro de perro que se ...
8,"El día 16 del mes pasado, entre las 12:30 y la...",¿Qué característica física particular tenían l...
9,5.º Si el Calendario no está a la venta para e...,¿Qué sucede si el Almanak no está disponible p...


In [ ]:
corpus_mixtral_expanded_10['respuesta'] = corpus_mixtral_expanded_10.apply(lambda row: generate_answer(row['frase'], row['pregunta_generada']), axis=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_

In [ ]:
corpus_mixtral_expanded_10['respuesta'][0]

'\n      Context information is below.\n      ---------------------\n      4.º El calendario debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más.\n      ---------------------\n      Given the context information and not prior knowledge, answer the query in Spanish.\n      Query: ¿Cuál es la fecha límite para que el Almanak esté impreso y disponible para la venta al público?\n      Answer:\n     La fecha límite para que el Almanak esté impreso y disponible para la venta al público es el 1 de noviembre de este año sin excepción.'

In [ ]:
corpus_mixtral_expanded_10.to_csv('respuestas_mixtral10.csv')

In [ ]:
def gen_respuesta(contexto,pregunta):
  query_engine = vector_index.as_query_engine(response_mode="compact")

  response = query_engine.query(f"Dado este contexto, {contexto} y esta pregunta {pregunta} genera una respuesta en español")

  pregunta = response.response#display_response(response)

  return pregunta

In [ ]:
corpus_mixtral_expanded_20 = corpus_mixtral_expanded[10:20]

In [ ]:
corpus_mixtral_expanded['respuesta'] = corpus_mixtral_expanded.apply(lambda row: gen_respuesta(row['frase'], row['pregunta_generada']), axis=1)

In [ ]:
corpus_mixtral_expanded.head()

,frase,pregunta_generada,respuesta
0,4.º El calendario debe ser impreso y estar a l...,¿Cuál es la fecha límite para que el Almanak e...,La fecha límite para que el Almanak esté impre...
1,4.º El calendario debe ser impreso y estar a l...,¿A qué precio se debe vender cada ejemplar imp...,Cada ejemplar impreso de dos pliegos debe vend...
2,4.º El calendario debe ser impreso y estar a l...,¿Cuál será la consecuencia si el Almanak no es...,La consecuencia de que el Almanak no esté disp...
3,En este discurso dedicado al Supremo Consejo y...,¿Qué se discute en el discurso dirigido al Sup...,Se discute en el discurso la importancia de la...
4,En este discurso dedicado al Supremo Consejo y...,¿Cuáles son los daños que se mencionan en el t...,Los daños que se mencionan en el texto que pue...


In [ ]:
corpus_mixtral_expanded.to_csv('respuestas_mixtral_v2.csv')

In [ ]:
context = corpus_mixtral_expanded_10['frase'][0]
question = corpus_mixtral_expanded_10['pregunta_generada'][0]

In [ ]:
prompt = f" Dado este texto: {context} y esta pregunta:\n{question}\n genera una respuesta"

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")

In [ ]:
outputs = model.generate(**inputs, max_length=500)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
outputs

tensor([[    1,   384,  1452,  7021,  2245, 28709, 28747, 28705, 28781, 28723,
         28936,  1744,  2300,   416,  3756,   340,  1105,  1201,  2824,   411,
         28709,   337, 27497,   264,   543,  8061, 28708,   389, 23912, 28709,
           639, 28705, 28740,   340, 26634,   340,  7021, 13680,  5450,   439,
         18316,  7909, 28723,  1343,  1105,   363,  2341,   331,   389,  4467,
           691,   340,   521,  1353,   340,  1326, 13170, 14150, 28324,   283,
          2824,   411, 28709,   340,  4882,   549, 11703,   385, 28725,   337,
           389,  4467,   691,   340,   268,  1296, 28706,  4014,   444,   385,
           639,   955,   934, 28797,   481, 10956, 28725,   337,   708,  5268,
         28723,   337, 12404,   710,  7429,  1100, 28747,    13, 28970, 28743,
         28718,  4304,  1037,   543,   701,  8136,   305,  9619,   570,  2646,
           955,   639,   976,  1294,   491,   934, 28797,  2824,   411, 28709,
           337, 25063,  1070,  2646,   543,  8061, 2

In [ ]:
outputs = model.generate(**inputs)

In [ ]:
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
answer

'Dado este texto: 4.º El calendario debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más. y esta pregunta:\n¿Cuál es la fecha límite para que el Almanak esté impreso y disponible para la venta al público?\n genera una respuesta así:\n La fecha límite para que el Almanak esté impreso y disponible para la venta al público es el 1 de noviembre de este año sin excepción.'

Otras pruebas

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import Settings

# Load data
reader = SimpleDirectoryReader("Manual0-100")
#reader = SimpleDirectoryReader(input_files=["essay.txt"])
documents = reader.load_data()

# Define LLM and embedding model
llm = MistralAI(api_key=api_key, model="mistral-medium")
embed_model = MistralAIEmbedding(model_name="mistral-embed", api_key=api_key)
Settings.llm = llm
Settings.embed_model = embed_model
# Create vector store index
index = VectorStoreIndex.from_documents(documents)

# Create query engine
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query(
    "What were the two main things the author worked on before college?"
)
print(str(response))